In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import SimpleITK as sitk
import mahotas as mt
import imageio

from PIL import Image
from matplotlib.patches import Rectangle
from scipy import linalg
from skimage.util.shape import view_as_windows
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import product
from skimage.feature.texture import greycomatrix
from collageradiomics import Collage, HaralickFeature, show_colored_image, InformationMeasureOfCorrelation1Interpretation, scale_array_for_image
from datetime import datetime

import os
import configparser

In [ ]:
# Read images
image_sitk = sitk.ReadImage('../../sample_data/ImageSlice.png')
image_array = sitk.GetArrayFromImage(image_sitk)
mask_image = sitk.ReadImage('../../sample_data/ImageNonRectangularMask.png')
mask_array = sitk.GetArrayFromImage(mask_image)

In [ ]:
collage = Collage(image_array, mask_array, verbose_logging=True)

In [ ]:
# Example of optional parameters
collage = Collage(
    image_array, 
    mask_array, 
    svd_radius=5, 
    verbose_logging=False, 
    haralick_feature_list=[HaralickFeature.All], 
    log_sample_rate=500, 
    cooccurence_angles=[0, 1*np.pi/4, 2*np.pi/4, 3*np.pi/4, 4*np.pi/4, 5*np.pi/4, 6*np.pi/4, 7*np.pi/4],
    information_measure_correlation1_interpretation = InformationMeasureOfCorrelation1Interpretation.XMinusYVariance,
    haralick_window_size=-1,
    greylevels=64
)

In [ ]:
# Run CoLlage Algorithm.
full_images = collage.execute()

In [ ]:
# Display gradient
figure, axes = plt.subplots(1, 2, figsize=(15, 15))
show_colored_image(figure, axes[0], collage.dx)
axes[0].set_title(f'Gx size={collage.dx.shape}')
show_colored_image(figure, axes[1], collage.dy)
axes[1].set_title(f'Gy size={collage.dy.shape}')

In [ ]:
# Display dominant angles
figure, axes = plt.subplots(1, 2, figsize=(15, 15))
show_colored_image(figure, axes[0], collage.dominant_angles_array)
axes[0].set_title('Dominant Angles (SVD)')
show_colored_image(figure, axes[1], collage.dominant_angles_shaped)
axes[1].set_title('Dominant Angles (Color Binned)')

In [ ]:
# Display dominant angles color-binned
figure, axis = plt.subplots(1,1, figsize=(5,5))
show_colored_image(figure, axis, collage.dominant_angles_shaped)

In [ ]:
# Display haralick
figure, axes = plt.subplots(3, 5, figsize=(15,15))

for row in range(3):
    for col in range(5):
        feature = row*5+col
        axis = axes[row][col]
        axis.set_axis_off()
        if feature>=13:
            continue
        show_colored_image(figure, axis, collage.haralick_features[:,:,feature])
        axis.set_title(f'Har {feature+1}')

In [ ]:
# Display full Haralick images with masked region.
aspect = image_array.shape[1] / image_array.shape[0]
figsize = 15 / aspect
figure, axes = plt.subplots(3, 5, figsize=(figsize * aspect, figsize))

for row in range(3):
    for col in range(5):
        feature = row*5+col
        axis = axes[row][col]
        axis.set_axis_off()
        if feature>=13:
            continue
        show_colored_image(figure, axis, collage.full_masked_images[feature])
        axis.set_title(f'Haralick {feature+1}')

In [ ]:
# Show preview of larger version of image.
plt.figure(figsize = (figsize * aspect, figsize))
figure = plt.imshow(collage.full_masked_images[0], cmap = plt.cm.jet)
figure.axes.get_xaxis().set_visible(False)
figure.axes.get_yaxis().set_visible(False)
plt.title('Haralick 1 Preview')

In [ ]:
import logging
logger = logging.getLogger('imageio').setLevel(logging.ERROR)

def scale_array_for_image(array_to_scale):
    flat_array = array_to_scale.flatten()
    minimum = float(min(flat_array))
    maximum = float(max(flat_array))
    array_range = maximum - minimum
    array_to_scale = array_to_scale - minimum
    array_to_scale /= array_range
    array_to_scale *= 255
    return array_to_scale

def write_haralick_image(feature, directory):
    haralick = scale_array_for_image(collage.haralick_features[:,:,feature])
    haralick = haralick.astype('uint8')
    imageio.imwrite(f'{directory}/haralick-feature-{feature + 1}.png', plt.cm.jet(haralick))
    img = Image.open(f'{directory}/haralick-feature-{feature + 1}.png')
    img = img.convert('RGBA')
    pixdata = img.load()

    width, height = img.size
    for y in range(height):
        for x in range(width):
            if pixdata[x, y] == (0, 0, 127, 255) or pixdata[x, y] == (127, 0, 0, 255):
                pixdata[x, y] = (255, 255, 255, 0)

    img.save(f'{directory}/haralick-feature-{feature + 1}.png', 'PNG')
    background = Image.open('../../sample_data/ImageSlice.png')
    foreground = Image.open(f'{directory}/haralick-feature-{feature + 1}.png')

    background = background.convert('RGBA')
    background.paste(foreground, (collage.mask_min_x, collage.mask_min_y), foreground)
    background.show()
    background.save(f'{directory}/haralick-feature-{feature + 1}.png')

In [ ]:
import pathlib

# Writes images to output_data directory in the root of the repository and show them.
print('Original Image')
display(Image.open('../../sample_data/ImageSlice.png'))

now = datetime.now()
date_string = now.strftime("%m%d%Y-%H%M%S")
output_directory = f'../../output_data/{date_string}'

pathlib.Path(output_directory).mkdir(parents=True, exist_ok=True) 

for feature in range(13):
    write_haralick_image(feature, output_directory)
    print(f'Haralick Feature {feature + 1}')
    display(Image.open(f'{output_directory}/haralick-feature-{feature + 1}.png'))